In [ ]:
from analysis.analysis import *
import matplotlib.pyplot as plt
import os

root = "corners_only_fixed_augmented"
gid = 3
save_path = f"../_server/sim_data/{root}/streamplots/grid {gid}/"
if not os.path.exists(save_path):
    os.makedirs(save_path)

Read a grid ID and aggregate across runs to compute $F$ and $\sigma$

In [ ]:
grid_df = read_fulltake(
    f"../_server/sim_data/{root}/parquets/fulltake_gid{gid}.parquet",
    scale_position=True,
)
grid_df = apply_time_filter(grid_df, dt=grid_df.iloc[0].add_rate)

nbins = 32
xva_df = get_xva_df(grid_df, nbins=nbins)


Plot of $F(x, v)$ with streamlines and $\sigma(x, v)$

In [ ]:
bounds, F, sigma = compute_F_sigma(xva_df, nbins=nbins, min_pts=10)

xmin, xmax = bounds["x"]
vmin, vmax = bounds["v"]
bounds_tup = (xmin, xmax, vmin, vmax, nbins)

plot_title = make_title(xva_df)
title = {"label": plot_title, "fontsize": 20}

init_pts = linear_lattice(xmin, xmax, vmin, vmax, n_pts=100)

fig, ax = plt.subplots(1, 2, figsize=(11, 3.5), dpi=300)

ax[0].set_title(**title)
F_streamplot(
    F,
    bounds_tup,
    stream_init_pts=init_pts,
    imshow_kwargs={"interpolation": "bilinear", "origin": "lower", "cmap": "jet"},
    streamplot_kwargs={
        "integration_direction": "forward",
        "color": "black",
        "broken_streamlines": False,
        "density": 1,
        "linewidth": 1.5,
    },
    do_try=True,
    ax=ax[0],
)

im = ax[1].imshow(
    sigma,
    extent=[xmin, xmax, vmin, vmax],
    interpolation="bilinear",
    origin="lower",
    cmap="viridis",
)
ax[1].set_xlabel(r"$x$ ($\mu$m)")
ax[1].set_ylabel(r"$v$ ($\mu$m/hr)")
cbar = plt.colorbar(im, ax=ax[1])
cbar.set_label(r"$\sigma$ ($\mu$m/hr$^{-3/2}$)")
ax[1].set_aspect("auto")

plt.subplots_adjust(wspace=0.5)
plt.show()

Plot $F(x\rightarrow 0, v)$

In [ ]:
X, Y, init_pts = full_lattice(F, xmin, xmax, vmin, vmax, nbins)

delta = 2
F_v_x0 = F[:, nbins // 2 - delta : nbins // 2 + (delta + 1)].mean(axis=1)

fig, ax = plt.subplots(1, 1, figsize=(3, 3), dpi=300)
mask = ~np.isnan(F_v_x0)
ax.plot(Y[mask, 0], F_v_x0[mask], "-o", c="black")
ax.set_xlabel(r"$v$ ($\mu$m/hr)")
ax.set_ylabel(r"$F(x\rightarrow 0)$ ($\mu$m/hr$^2$)")
plt.show()


Phase separation for bistable cases

In [ ]:
colors = []

for xx, yy in init_pts:
    fig, ax = plt.subplots(1, 1)
    try:
        stream = ax.streamplot(
            X,
            Y,
            Y,
            F,
            linewidth=0.5,
            start_points=[[xx, yy]],
            integration_direction="forward",
            color="black",
            broken_streamlines=False,
            density=1,
        )
        streamlines = np.array(stream.lines.get_segments())
        plt.close(fig)

        if len(streamlines) == 0:
            colors.append("gainsboro")
            continue

        end_pt = streamlines[-1][-1]
        if end_pt[0] < 133:
            colors.append("salmon")
        elif end_pt[0] > 167:
            colors.append("cornflowerblue")
        else:
            colors.append("wheat")

    except ValueError:
        colors.append("gainsboro")
        plt.close(fig)
        continue


In [ ]:
import matplotlib.patches as mpatches

from importlib import reload
from analysis import analysis

reload(analysis)
from analysis.analysis import *

legend_dict = {
    "salmon": mpatches.Patch(color="salmon", label="Left"),
    "cornflowerblue": mpatches.Patch(color="cornflowerblue", label="Right"),
    "wheat": mpatches.Patch(color="wheat", label="Middle"),
}

legend = []
for c in np.unique(colors):
    legend.append(legend_dict[c]) if c != "gainsboro" else None

plt.figure(figsize=(3, 3), dpi=300)
plt.vlines(
    x=133, ymin=vmin, ymax=vmax, linewidth=1, colors=["black"], linestyles=["dashed"]
)
plt.vlines(
    x=167, ymin=vmin, ymax=vmax, linewidth=1, colors=["black"], linestyles=["dashed"]
)

dx = (xmax - xmin) / nbins
dv = (vmax - vmin) / nbins
boundary_pts = get_boundary_points(init_pts, colors)
# sorted_pts = break_upper_lower(boundary_pts)
sorted_pts = TSP(boundary_pts, dx, dv)[:-4]

plt.legend(handles=legend, bbox_to_anchor=(1.01, 1))
plt.scatter(init_pts[:, 0], init_pts[:, 1], color=colors, s=5)
plt.plot(sorted_pts[:, 0], sorted_pts[:, 1], c="black", lw=2.5)
# plt.scatter(sorted_pts[:, 0], sorted_pts[:, 1], c="black", s=10)
plt.xlabel(r"$x$ ($\mu$m)")
plt.ylabel(r"$v$ ($\mu$m/hr)")
xlim = plt.xlim()
ylim = plt.ylim()
plt.show()


3. View stream line bunches per each category of end point behavior

In [ ]:
red_indx = np.where(np.array(colors) == "red")[0]
blue_indx = np.where(np.array(colors) == "blue")[0]
orange_indx = np.where(np.array(colors) == "orange")[0]


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 3), dpi=300)
stream_colors = {"red": "lightcoral", "blue": "lightsteelblue", "orange": "wheat"}

for indx in [red_indx, blue_indx, orange_indx]:
    if len(indx) == 0:
        continue

    starting_pts = init_pts[indx]
    c = colors[indx[0]]
    stream_c = stream_colors[c]

    for xx, yy in starting_pts:
        stream = ax.streamplot(
            X,
            Y,
            Y,
            F,
            linewidth=1,
            start_points=[[xx, yy]],
            integration_direction="forward",
            color=stream_c,
            broken_streamlines=False,
            density=1,
        )
        streamlines = np.array(stream.lines.get_segments())

        if len(streamlines) == 0:
            continue

        start_pt = streamlines[0][0]
        end_pt = streamlines[-1][-1]
        ax.scatter(*end_pt, s=50, color=c, marker="*", zorder=5, label="end")
        ax.set_xlim(bounds[0:2])
        ax.set_ylim(bounds[2:4])
        ax.set_xlabel(r"$x$ ($\mu$m)")
        ax.set_ylabel(r"$v$ ($\mu$m/hr)")
        # ax.legend(bbox_to_anchor=(1.7, 1))
plt.savefig(save_path + f"lattice_segments.png")
plt.close()
# plt.show()
